In [1]:
from bayesian_network import BayesianNetwork 
import numpy as np

In [2]:
net = BayesianNetwork(values=["True", "False"]) 


net.add_node("A")
net.add_node("B")
net.add_node("C")
net.add_edge("A", ["B"])

In [3]:
print(net)
net.check_valid()

Nodes: dict_keys(['A', 'B', 'C']), Edges: {'A': ['B']}
Node B probability table doesn't match parents values


False

In [4]:
net.add_edge("B", ["C"])
print(net)


Nodes: dict_keys(['A', 'B', 'C']), Edges: {'A': ['B'], 'B': ['C']}


In [5]:
net.check_valid()

Node B probability table doesn't match parents values
Node C probability table doesn't match parents values


False

In [6]:
net.add_prob_table("A", {"Prior" : [0.5, 0.5]})
net.add_prob_table("B", {"True" : [0.8, 0.2], "False" : [0.2, 0.8]})
net.add_prob_table("D", {"Prior" : [0.9, 0.1]})

In [7]:
net.add_node("D")
net.add_edge("D", ["C"])
net.add_prob_table("C", {("True", "True") : [0.7, 0.3],
                          ("True", "False") : [0.3, 0.7],
                            ("False", "True") : [0.1, 0.9],
                            ("False", "False") : [0.9, 0.1]})
                          
print(net)
net.check_valid()

Nodes: dict_keys(['A', 'B', 'C', 'D']), Edges: {'A': ['B'], 'B': ['C'], 'D': ['C']}


True

In [8]:
probs = net.get_probabilities("A")
print(probs)
for i in range(30):

    print(net.sample(probs))

[0.5, 0.5]
0
1
1
0
0
0
1
1
0
0
1
1
0
0
0
1
0
0
0
0
1
0
1
0
1
1
1
1
0
0
